In [ ]:
import numpy as np
import random
import gym
import time
import timeit

class PolicyGradRBF(object):
    
    def __init__(self, __env, __actions, __stateCenters, __actionCenters, __stateSigma, __actionSigma):
        self.env = __env
        self.actions = __actions
        self.stateCenters = __stateCenters
        self.actionCenters = __actionCenters
        self.stateSigma = __stateSigma
        self.actionSigma = __actionSigma
        
        self.numActions = len(__actions)
        self.numStateCenters = len(__stateCenters)
        self.numActionCenters = len(__actionCenters)
        
        
        self.teta = np.zeros((self.numActionCenters,1))
        self.w = np.zeros((self.numStateCenters,1))
        self.actionPhi = np.zeros((self.numActionCenters))
        self.statePhi = np.zeros((self.numStateCenters))
        
        
    def createRBFState(self, state, sigma, centers):
        center = np.array([centers[0], centers[1]])
        
        state = np.array([(state[0]+1.2)/1.7, (state[1]+0.07)/0.14])
        
        dist = (state - center).reshape(2,1)
        distT = dist.reshape(1,2)
        b = distT.dot(sigma)
        b = b.dot(dist)
        
        return np.exp(-b*b)
    
    def createRBFAction(self, state, sigma, centers, action):
        center = np.array([centers[0], centers[1], centers[2]])
        
        state = np.array([(state[0]+1.2)/1.7, (state[1]+0.07)/0.14, action])
        
        dist = (state - center).reshape(3,1)
        distT = dist.reshape(1,3)
        b = distT.dot(sigma)
        b = b.dot(dist)
        
        return np.exp(-b*b)

    def createActionPhi(self, state, action):
        self.actionPhi = np.zeros((self.numActionCenters))
        for i in range(self.numActionCenters):
            self.actionPhi[i] = self.createRBFAction(state, self.actionSigma, self.actionCenters[i], action)
            
    def createStatePhi(self, state):
        self.statePhi = np.zeros((self.numStateCenters))
        for i in range(self.numStateCenters):
            self.statePhi[i] = self.createRBFState(state, self.stateSigma, self.stateCenters[i])
            
    def createStateActionValue(self, state, action):
        self.createActionPhi(state, action)
        return self.teta.T.dot(self.actionPhi)
    
    def createStateActionsValues(self, state):
        values = []
        
        for action in self.actions:
            values.append(self.createStateActionValue(state, action))
            
        return values
    
    def calcPref(self, state):
        stateActionsValues = self.createStateActionsValues(state)
        pref = []
        
        for value in stateActionsValues:
            pref.append(np.exp(value))
            
        return pref
    
    def calcProbActions(self, state):
        pref = self.calcPref(state)
        normFactor = sum(pref)
        return [float(num)/normFactor for num in pref]
    
    def chooseBestAction(self, state):
        stateActionsValues = self.createStateActionsValues(state)
        prob = self.calcProbActions(state)
        index = np.random.choice(np.arange(len(stateActionsValues)), 1, prob)
        
        return self.actions[index[0]]
    
    def calcStateValue(self, state):
        self.createStatePhi(state)
        
        statePhiT = self.statePhi.reshape(self.numStateCenters,1)
        
        return self.w.T.dot(statePhiT)
    
    def calcPolicy(self, state, action):
        stateActionValue = self.createStateActionValue(state, action)
        pref = self.calcPref(state)
        normFactor = sum(pref)
        
        return np.exp(stateActionValue)/normFactor
    
    def calcGradPolicy(self, state, action):
        const = 0
        for iterAction in self.actions:
            self.createActionPhi(state, iterAction)
            const += self.calcPolicy(state, action) * self.actionPhi
            
        self.createActionPhi(state, action)
        return (self.actionPhi-const).reshape(self.numActionCenters,1)
    
    def updateW(self, state, alphaW, delta, gammaNew):
        self.createStatePhi(state)
        statePhiT = self.statePhi.reshape(self.numStateCenters,1)
        self.w += alphaW * delta * gammaNew * statePhiT
    
    def updateTeta(self, alphaTeta, delta, gammaNew, gradPolicy):
        self.teta += alphaTeta * gammaNew * delta * gradPolicy

            
    def calcDelta(self, nextState, currState, reward, gamma):
        currStateValue = self.calcStateValue(currState)
        nextStateValue = self.calcStateValue(nextState)
        
        return reward + gamma * nextStateValue - currStateValue
    
    def doAlgorithm(self, gamma, alphaTeta, alphaW):
        
        currState = self.env.reset()
        currAction = self.chooseBestAction(currState)
        gammaNew = 1  
        while currState[0] < 0.5:
            self.env.render()
            
            nextState, reward, done, info = self.env.step(currAction)
            
            if nextState[0] > 0.49:
                reward = 200

            nextAction = self.chooseBestAction(nextState)
            
            delta = self.calcDelta(nextState, currState, reward, gamma)
            
            self.updateW(currState, alphaW, delta, gammaNew)
            gradPolicy = self.calcGradPolicy(currState, currAction)
            self.updateTeta(alphaTeta, delta, gammaNew, gradPolicy)
            
            currState = nextState
            currAction = nextAction
            gammaNew = gammaNew * gamma
            

env = gym.make("MountainCar-v0")

positionCenter = np.linspace(-1.2, 0.6, 5)
velocityCenter = np.linspace(-0.07, 0.07, 3)
actionCentersVal = [0.5, 1.5]

stateCenters = []
actionCenters = []
for posCent in positionCenter:
    for velCent in velocityCenter:
        stateCenters.append((posCent,velCent))
        for actCenter in actionCentersVal:
            actionCenters.append((posCent,velCent, actCenter))

actions = [0,1,2]  

stateSigma = np.zeros((2,2))
stateSigma[0,0] = 1
stateSigma[1,1] = 1

actionSigma = np.zeros((3,3))
actionSigma[0,0] = 1
actionSigma[1,1] = 1
actionSigma[2,2] = 1

gamma = 0.9
alphaW = 0.1/50
alphaTeta = 0.1/50

start_time = timeit.default_timer()
myalgo = PolicyGradRBF(env, actions, stateCenters, actionCenters, stateSigma, actionSigma)
myalgo.doAlgorithm(gamma, alphaTeta, alphaW)
elapsed_time = timeit.default_timer() - start_time
print(' \n Reached the top in {:.2f}s '.format(elapsed_time))